# Transfer Learning with Full and Last Layer Update

This hands-on trains a neural network model to classify images of clothing, like sneakers and shirts, from the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset.

**Goals:**

*   Train a vanilla Fashion MNIST classifier. Analyse its performance on a dataset which suffers from a distribution shift. We will consider rotation transformation to simulate distribution shift in input data.
*   Use transfer learning to light re-train a trained model to adapt it to a different degree than it was initially trained for.
*   Use transfer learning to re-train only the last layers of the model to adapt it to a different degree than it was initially trained for.
*   Train a joint model for all degrees. Compare the results.



In [ ]:
import tensorflow as tf
from tensorflow import keras

import datetime

import math
import numpy as np
import random
import matplotlib.pyplot as plt
plt.style.use('default')

random.seed(42)
np.random.seed(42)

print(tf.__version__)

Fashion MNIST dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels). We use Fashion MNIST because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

We will use 60,000 images to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow, just import and load the data.

Loading the dataset returns four NumPy arrays:

*   The train_images and train_labels arrays are the training set—the data the model uses to learn.
*   The model is tested against the test set, the test_images, and test_labels arrays.

The images are 28x28 NumPy arrays, with pixel values ranging between 0 and 255. The labels are an array of integers, ranging from 0 to 9.

Each image is mapped to a single label. Since the class names are not included with the dataset, store them here to use later when plotting the images.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

**Exercise:** Explore the format of the dataset before training the model.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

**Exercise:** Display the first 25 images from the training set and display the class name below each image. Verify that the data is in the correct format and we're ready to build and train the network.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255.

In [ ]:
print(train_images[0])

**Exercise:** Scale these values to a range of 0 to 1 before feeding to the neural network model. For this, you need to divide the values by 255. It's important that the training set and the testing set are preprocessed in the same way:

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

## Build a model

Building the neural network requires configuring the layers of the model, then compiling the model. Most of deep learning consists of chaining together simple layers. Most layers, like `tf.keras.layers.Dense`, have parameters that are learned during training.

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(6, (5, 5), padding='same', activation='relu',kernel_initializer='he_uniform', input_shape=(28, 28,1)),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(32, (5, 5), kernel_initializer='he_uniform', activation='relu'),
  keras.layers.Dropout(0.1),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  # keras.layers.Dropout(0.1, name="baselayer"),
  keras.layers.Dropout(0.1),
  keras.layers.Conv2D(120, (5, 5), kernel_initializer='he_uniform', activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(84, activation='relu', name="baselayer"),
  keras.layers.Dense(10, activation=tf.nn.softmax)
])

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

*   Loss function = measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
*   Optimizer = this is how the model is updated based on the data it sees and its loss function.
*   Metrics = used to monitor the training and testing steps. The following example uses `accuracy`, the fraction of the images that are correctly classified.

**Exercise:** Print model summary.

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### START CODE HERE ###

### END CODE HERE ###

Training the neural network model requires the following steps:

*   Feed the training data to the model — in this example, the train_images and train_labels arrays.
*   The model learns to associate images and labels.
*   We ask the model to make predictions about a test set—in this example, the test_images array. We verify that the predictions match the labels from the test_labels array.

**Exercise:** To start training, call the `model.fit` method—the model is "fit" to the training data. Use `epochs`=10, `validation_split`=0.1 and `batch_size`=32.

In [ ]:
### START CODE HERE ###
history = None
### END CODE HERE ###

## Evaluate Model

In [ ]:
plt.rcParams["figure.figsize"] = (6,4)

# graph the loss, the model above is configure to use "mean squared error" as the loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g-', label='Training loss')
plt.plot(epochs, val_loss, 'b-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (6,4)
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(epochs[:], accuracy[:], 'g-', label='Training accuracy')
plt.plot(epochs[:], val_accuracy[:], 'b-', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


**Exercise:** Compute how the model performs on the test dataset.

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

In [ ]:
plt.rcParams["figure.figsize"] = (20,4)

# use the model to predict the test inputs
predictions = model.predict(test_images)

# print the predictions and the expected ouputs
print("predictions =\n", np.argmax(predictions, axis=1))
print("actual =\n", test_labels)

# Plot the predictions along with to the test data
plt.title('Training data predicted vs actual values')
plt.plot(test_labels[:300], 'bs', label='Actual')
plt.plot(np.argmax(predictions[:300], axis=1), 'ro', label='Predicted')
plt.show()

Plot confusion matrix

In [ ]:
confusion = tf.math.confusion_matrix(labels=tf.constant(test_labels.flatten()),
                                       predictions=tf.constant(np.argmax(predictions, axis=1)),
                                       num_classes=len(class_names))

import seaborn as sns
plt.figure(figsize=(6, 4))
sns.heatmap(confusion,
            xticklabels=class_names,
            yticklabels=class_names,
            annot=True, cmap='Blues', fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

print("Loss {}, Accuracy {}".format(test_loss, test_acc))

## Rotate Dataset

In [ ]:
!pip install -U tensorflow-addons

**Exercise:** Plot an image from the test set before and after rotation.

**Exercise:** Compute model accuracy in the test set before and after rotation.

In [ ]:
import tensorflow_addons as tfa

def rotate_dataset(dataset, degree):
  # need to reshape to match the expected NHWC input format. See: https://www.tensorflow.org/addons/api_docs/python/tfa/image/rotate
  result = tfa.image.rotate(dataset.reshape(dataset.shape[0], dataset.shape[1], dataset.shape[2], 1),
                                          tf.constant(degree * math.pi / 180), interpolation= "nearest")
  result = result.numpy().reshape(dataset.shape[0], dataset.shape[1], dataset.shape[2])
  return result

degree = 30
test_images_rotated = rotate_dataset(test_images, degree)

plt.rcParams["figure.figsize"] = (6,4)
### START CODE HERE ###

### END CODE HERE ###


### START CODE HERE ###

### END CODE HERE ###


## Evaluate on All Degrees

In [ ]:
acc = []

for d in range(0,360,10):
  r = rotate_dataset(test_images, d)
  _, tacc = model.evaluate(r, test_labels, verbose=1)
  print(tacc)
  acc = np.append(acc, tacc)

In [ ]:
theta = np.arange(0, 360, 10)/180 * math.pi
ref = np.ones_like(theta)*test_acc

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, ref, label='Reference')
ax.plot(theta, acc, label='Vanilla')
ax.set_rmin(0)
ax.set_theta_zero_location("N")
ax.set_rmax(1.0)
ax.grid(True)

ax.legend()
ax.set_title("Accuracy", va='bottom')
plt.show()

## Freeze the Model

In [ ]:
model.trainable = False

model.summary()

# Transfer Learning. Re-training the model for X degree. Update all parameters.

In [ ]:
degree = 10

train_images2 = rotate_dataset(train_images, degree)

test_loss_rotated, test_acc_rotated = model.evaluate(train_images2, train_labels, verbose=0)
print('Test accuracy on the original model:', test_acc_rotated)

model2 = keras.models.clone_model(model)
model2.trainable = True
print(model2.summary())

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history2 = model2.fit(train_images2, train_labels, epochs=10, validation_split=0.1, batch_size=32)

In [ ]:
test_images_rotated = rotate_dataset(test_images, degree)

# old model
print("model:")
_, test_acc_rotated = model.evaluate(test_images,  test_labels, verbose=0)
print('Test accuracy 0 degrees:', test_acc_rotated)

_, test_acc_rotated = model.evaluate(test_images_rotated,  test_labels, verbose=0)
print('Test accuracy rotated:', test_acc_rotated)

# new model2
print("model2:")
_, test_acc_rotated = model2.evaluate(test_images,  test_labels, verbose=0)
print('Test accuracy 0 degrees:', test_acc_rotated)

_, test_acc_rotated = model2.evaluate(test_images_rotated,  test_labels, verbose=0)
print('Test accuracy rotated:', test_acc_rotated)

In [ ]:
acc2 = []

for d in range(0,360,10):
  r = rotate_dataset(test_images, d)
  _, tacc = model2.evaluate(r, test_labels, verbose=1)
  print(tacc)
  acc2 = np.append(acc2, tacc)


In [ ]:
theta = np.arange(0, 360, 10)/180 * math.pi
ref = np.ones_like(theta)*test_acc

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, ref, label='Reference')
ax.plot(theta, acc, label='Vanilla')
ax.plot(theta, acc2, label='TL Full')
ax.set_rmin(0)
ax.set_theta_zero_location("N")
ax.set_rmax(1.0)
ax.grid(True)

ax.legend()
ax.set_title("Accuracy", va='bottom')
plt.show()

# Transfer Learning. Re-training the model for X degree. Update head.

In [ ]:
# chop off model's head
base = keras.Model(model.input, model.get_layer('baselayer').output)
print(base.summary())

x = base.output
# x = keras.layers.Conv2D(120, (5, 5), kernel_initializer='he_uniform', activation='relu')(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(84, activation='relu')(x)
head = keras.layers.Dense(10, tf.nn.softmax)(x)
model3 = keras.Model(inputs = base.input, outputs = head)
model3.summary()

In [ ]:
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
degree = 10

train_images3 = rotate_dataset(train_images, degree)

_, test_acc_rotated = model.evaluate(train_images3, train_labels, verbose=0)
print('Test accuracy on the original model:', test_acc_rotated)

model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history3 = model3.fit(train_images3, train_labels, epochs=10, validation_split=0.1, batch_size=32)

In [ ]:
test_images_rotated = rotate_dataset(test_images, degree)

# old model
print("model:")
_, test_acc_rotated = model.evaluate(test_images,  test_labels, verbose=0)
print('Test accuracy 0 degrees:', test_acc_rotated)

_, test_acc_rotated = model.evaluate(test_images_rotated,  test_labels, verbose=0)
print('Test accuracy rotated:', test_acc_rotated)

# new model3
print("model3:")
_, test_acc_rotated = model3.evaluate(test_images,  test_labels, verbose=0)
print('Test accuracy 0 degrees:', test_acc_rotated)

_, test_acc_rotated = model3.evaluate(test_images_rotated,  test_labels, verbose=0)
print('Test accuracy rotated:', test_acc_rotated)

In [ ]:
acc3 = []

for d in range(0,360,10):
  r = rotate_dataset(test_images, d)
  _, tacc = model3.evaluate(r, test_labels, verbose=1)
  print(tacc)
  acc3 = np.append(acc3, tacc)


In [ ]:
theta = np.arange(0, 360, 10)/180 * math.pi
ref = np.ones_like(theta)*test_acc

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, ref, label='Reference')
ax.plot(theta, acc, label='Vanilla')
ax.plot(theta, acc2, label='TL Full')
ax.plot(theta, acc3, label='TL Last Layer')
ax.set_rmin(0)
ax.set_theta_zero_location("N")
ax.set_rmax(1.0)
ax.grid(True)

ax.legend()
ax.set_title("Accuracy", va='bottom')
plt.show()

# Train a Model for All Degrees

In [ ]:
model4 = keras.Sequential([
  # factor=(-0.5,0.5) = +- 180 degrees
  keras.layers.RandomRotation(factor=(-0.5,0.5), interpolation= "nearest", input_shape=(28, 28,1)),
  keras.layers.Conv2D(6, (5, 5), padding='same', activation='relu',kernel_initializer='he_uniform', input_shape=(28, 28,1)),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(32, (5, 5), kernel_initializer='he_uniform', activation='relu'),
  keras.layers.Dropout(0.1),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Dropout(0.1, name="baselayer"),
  keras.layers.Conv2D(120, (5, 5), kernel_initializer='he_uniform', activation='relu'),
  keras.layers.Flatten(),
  keras.layers.Dense(84, activation='relu'),
  keras.layers.Dense(10, activation=tf.nn.softmax)
])

model4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history4 = model4.fit(train_images, train_labels, epochs=10, validation_split=0.1, batch_size=32)
print(model4.summary())

In [ ]:
acc4 = []

for d in range(0,360,10):
  r = rotate_dataset(test_images, d)
  _, tacc = model4.evaluate(r, test_labels, verbose=1)
  print(tacc)
  acc4 = np.append(acc4, tacc)


In [ ]:
theta = np.arange(0, 360, 10)/180 * math.pi
ref = np.ones_like(theta)*test_acc

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, ref, label='Reference')
ax.plot(theta, acc, label='Vanilla')
ax.plot(theta, acc2, label='TL Full')
ax.plot(theta, acc3, label='TL Last Layer')
ax.plot(theta, acc4, label='All degrees')
ax.set_rmin(0)
ax.set_theta_zero_location("N")
ax.set_rmax(1.0)
ax.grid(True)

ax.legend()
ax.set_title("Accuracy", va='bottom')
plt.show()